In [23]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sbn
import pandas as pd
import datetime
import time
from pandas.api.types import CategoricalDtype

%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
plt.style.use('default') 
sbn.set(style="whitegrid") 

# **Satinización de archivos de edad y genero**

In [43]:
#Antes que nada, me intriga si los archivos de 'datos_navent' y los de 'hasta15/4' tienen overlap, son redundantes o consecutivos
genero_edad_navent = pd.read_csv('datos/datos_navent/fiuba_2_postulantes_genero_y_edad.csv')
genero_edad_hasta_15_4 = pd.read_csv('datos/hasta_15_4/fiuba_2_postulantes_genero_y_edad.csv')
genero_edad_desde_15_4 = pd.read_csv('datos/desde_15_4/fiuba_2_postulantes_genero_y_edad.csv')
genero_edad_postulantes = genero_edad_navent.merge(genero_edad_hasta_15_4, on=['idpostulante','fechanacimiento','sexo'], how='outer')

In [27]:
print("tamaño archivo navent {}".format(genero_edad_navent.size))
print("tamaño archivo hasta_15_4 {}".format(genero_edad_hasta_15_4.size))
print("tamaño archivo desde_15_4 {}".format(genero_edad_desde_15_4.size))
print("tamaño archivo merge postulantes {}".format(genero_edad_postulantes.size))
def postulantes_str2timestamp(s): # debo
    if type(s) is datetime.datetime:
        return s
    try:
        return datetime.datetime.strptime(s, '%Y-%m-%d')
    except:
        return None
print (genero_edad_postulantes.describe()) #veo que hay repetidos por id
print (genero_edad_postulantes['sexo'].value_counts()) #veo que hay gente (4) con sexo '0'
genero_edad_postulantes.dtypes
# Genero una funcion para satinizar archivos de postulantes_genero_edad
genero_categoria = CategoricalDtype(categories=["MASC","FEM","NO_DECLARA"], ordered=False)
def satinize_postulante_genero_edad(df):
    df['sexo'] = df['sexo'].astype(genero_categoria).fillna("NO_DECLARA")
    df = df.drop_duplicates(subset='idpostulante')
    df['fechanacimiento']= df['fechanacimiento'].map(postulantes_str2timestamp)
    return df
genero_edad_postulantes = satinize_postulante_genero_edad(genero_edad_postulantes)
genero_edad_postulantes.head()

tamaño archivo navent 602664
tamaño archivo hasta_15_4 893241
tamaño archivo desde_15_4 844155
tamaño archivo merge postulantes 1224438
       idpostulante      fechanacimiento    sexo
count        408146               388502  408146
unique       408146                16866       3
top         5mq85Zv  1991-06-20 00:00:00     FEM
freq              1                   89  204651
FEM           204651
MASC          184467
NO_DECLARA     19028
Name: sexo, dtype: int64


,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03 00:00:00,FEM
1,5awk,1962-12-04 00:00:00,FEM
2,ZaO5,1978-08-10 00:00:00,FEM
3,NdJl,1969-05-09 00:00:00,MASC
4,eo2p,1981-02-16 00:00:00,MASC


## postulaciones

In [41]:
postulaciones = pd.read_csv(
    'datos/datos_navent/fiuba_4_postulaciones.csv'
).merge(pd.read_csv('datos/hasta_15_4/fiuba_4_postulaciones.csv'))
print(postulaciones.size)
print(postulaciones.drop_duplicates().size) #no hay duplicados!
print(postulaciones.head())
def sanitize_postulaciones(df):
    postulaciones['date'] = pd.to_datetime(postulaciones['fechapostulacion']).dt.date
    postulaciones['time'] = pd.to_datetime(postulaciones['fechapostulacion']).dt.time
    return postulaciones.copy()
postulaciones = sanitize_postulaciones(postulaciones)
print(postulaciones.head())

5120190
5120190
      idaviso idpostulante     fechapostulacion
0  1112286523         ZaO5  2018-01-24 15:07:39
1  1112272060         ZaO5  2018-01-24 15:20:10
2  1112288401         ZaO5  2018-01-26 08:37:04
3  1112300563         ZaO5  2018-01-30 13:35:48
4  1112293018         ZaO5  2018-01-31 08:41:05
      idaviso idpostulante     fechapostulacion        date      time
0  1112286523         ZaO5  2018-01-24 15:07:39  2018-01-24  15:07:39
1  1112272060         ZaO5  2018-01-24 15:20:10  2018-01-24  15:20:10
2  1112288401         ZaO5  2018-01-26 08:37:04  2018-01-26  08:37:04
3  1112300563         ZaO5  2018-01-30 13:35:48  2018-01-30  13:35:48
4  1112293018         ZaO5  2018-01-31 08:41:05  2018-01-31  08:41:05


## Archivo 'avisos_detalle'

In [77]:
avisos_detalle = pd.read_csv(
    'datos/datos_navent/fiuba_6_avisos_detalle.csv'
).merge(pd.read_csv('datos/hasta_15_4/fiuba_6_avisos_detalle.csv'))
avisos_detalle.isnull().any()
#ESTE DF TIENE MUCHOS DATOS NULOS EN LAS COLUMNAS DE 'CIUDAD' y 'MAPACALLE'. 
#Decidimos eliminarlas ya que no nos parecieron muy relevantes para el analisis
#idpais solo tiene valor 1, la descripcion nunca es nula
#avisos_detalle.drop('descripcion',1,inplace=True)
tipo_trabajo = CategoricalDtype(
    categories=["Full-time","Part-time","Teletrabajo","Por Horas","Pasantia","Temporario","Por Contrato","Fines de Semana","Primer empleo"],
    ordered=True #de 'mas fijo' a 'menos fijo'
)
avisos_detalle.nombre_area.value_counts()# hay muchas areas, no van a tener su propia categoria
def sanitize_aviso_detalle(df):
    df['tipo_de_trabajo'] = df['tipo_de_trabajo'].astype(tipo_trabajo)
    return df.drop(['ciudad','idpais','mapacalle'],1)
avisos_detalle = sanitize_aviso_detalle(avisos_detalle)

## Archivo 'educacion_postulantes'

In [69]:
educacion_postulantes = pd.read_csv(
    'datos/datos_navent/fiuba_1_postulantes_educacion.csv'
).merge(pd.read_csv('datos/hasta_15_4/fiuba_1_postulantes_educacion.csv'))
educacion_postulantes.isna().any() #no hay nulos
educacion_postulantes.duplicated().value_counts()#ni duplicados

idpostulante    False
nombre          False
estado          False
dtype: bool

# Genero nivel_educativo, completando el dataframe de postulantes

In [110]:
def generar_nivel_educativo(postulantes, educacion):
    #Considero como nivel educativo el maximo nivel que alcanzaron que fue completado.
    #Descarto estudios en curso o abandonados, quizas deba volver aca en algun momento
    tipo_estudio = CategoricalDtype(
        categories=["Otro","Secundario","Terciario/Técnico","Universitario","Posgrado","Master","Doctorado"],
        ordered=True
    )
    educacion = educacion[educacion.estado=='Graduado'].drop(['estado'],1)
    educacion.nombre= educacion.nombre.astype(tipo_estudio)
    educacion = educacion.sort_values(by='nombre',ascending=True)
    educacion = educacion.drop_duplicates(subset='idpostulante',keep='last')
    postulantes = postulantes.merge(educacion, on='idpostulante')
    postulantes = postulantes.rename(index=str,columns={'nombre':'nivel_educativo'})
    return postulantes.copy()
postulantes = generar_nivel_educativo(genero_edad_postulantes, educacion_postulantes)

,idpostulante,fechanacimiento,sexo,nivel_educativo
0,ZaO5,1978-08-10,FEM,Terciario/Técnico
1,NdJl,1969-05-09,MASC,Universitario
2,8BkL,1976-07-21,FEM,Master
3,NPBx,1981-03-09,MASC,Universitario
4,YGBY,1963-04-09,FEM,Terciario/Técnico
5,6aGl,1967-01-30,FEM,Posgrado
6,10bb,1954-08-30,MASC,Universitario
7,eWO2,1959-01-05,MASC,Posgrado
8,6Kbl,1976-09-03,MASC,Master
9,ZqWb,1971-11-08,MASC,Master


In [ ]:
# Hay ids de postulantes duplicados, lo que es consistente con que un postulante tenga en su perfil
# por ejemplo, un estudio universitario en curso y un estudio secundario completo
educacion_postulantes.duplicated(subset = 'idpostulante').any()

## Archivo 'vistas'

In [ ]:
vistas.head()
# Plantea la relación entre postulantes y avisos

In [ ]:
vistas.isnull().any()

In [ ]:
vistas.duplicated().any()

## Arichivo 'avisos_online'

In [ ]:
avisos_online.isnull().any()

In [ ]:
avisos_online.duplicated().any()

# Algunas exploraciones de cada archivo por separado

In [ ]:
#Distribución de las postulaciones por dia del mes
g_day = pd.to_datetime(postulaciones['date']).dt.day.hist(bins=31,edgecolor='black');
g_day.set_title('Disitribucion de las postulaciones por día del mes',fontsize=15)
g_day.set_xlabel('Dia')
g_day.set_ylabel('Cantidad de postulaciones');

In [ ]:
# ¿Será que uno se pone a buscar trabajo cuando se le empieza a acabar el sueldo?
g_day = pd.to_datetime(postulaciones['date']).dt.day.hist(bins=4,edgecolor='black');

In [ ]:
enero = postulaciones['date'][pd.to_datetime(postulaciones['date']).dt.month == 1]
enero = pd.DataFrame(enero)

febrero = postulaciones['date'][pd.to_datetime(postulaciones['date']).dt.month == 2]
febrero = pd.DataFrame(febrero)

sizes = [enero.count(),febrero.count()]
names = ['Enero', 'Febrero']

plt.figure(figsize=(3, 3))
plt.title('Distribucion de las postulaciones por mes', fontsize=15)
plt.pie(sizes, labels=names, autopct='%1.1f%%', startangle=90, colors=['green', 'lightblue'], explode=(0.1, 0),shadow=True)
plt.show()

In [ ]:
# ¡Los datos empiezan a partir del 15/1!
postulaciones['date'].min()

Ambos graficos son engañosos.
El primero muestra que hay mas postulaciones en la segunda mitad de los meses pero eso es porque los datos de Enero son solo de la segunda mitad del mes.
De igual manera el sugundo grafico muestra que Enero tiene practicamente la mitad de postulaciones que Febrero.
Tambien se debe a lo comentado anteriormente, solo hay datos de la segunda mitad de Enero.

In [ ]:
#Febrero tiene (casi)los dias completos
febrero = postulaciones['date'][pd.to_datetime(postulaciones['date']).dt.month == 2]
febrero = pd.DataFrame(febrero.unique()).sort_values(by=0, ascending=False)
febrero

In [ ]:
g_month_enero = pd.to_datetime(postulaciones['date'][pd.to_datetime(postulaciones['date']).dt.month == 1]).dt.day.hist(bins=17,edgecolor='black');
g_month_enero.set_title('Distribucion de postulaciones durante Enero',fontsize=15)
g_month_enero.set_xlabel('Dias')
g_month_enero.set_ylabel('Cantidad de postulaciones');

In [ ]:
g_month_febrero = pd.to_datetime(postulaciones['date'][pd.to_datetime(postulaciones['date']).dt.month == 2]).dt.day.hist(bins=28,edgecolor='black');
g_month_febrero.set_title('Distribucion de postulaciones durante Febrero',fontsize=15)
g_month_febrero.set_xlabel('Dias')
g_month_febrero.set_ylabel('Cantidad de postulaciones');

Ambos gráficos presentan 'bajones' de 2-3 días de duración: ¿Son fines de semana

In [ ]:
postulaciones['weekday']= pd.to_datetime(postulaciones['date']).dt.dayofweek
days = ['lunes','martes','miercoles','jueves','viernes','sabado','domingo']
postulaciones['weekday'] = pd.Categorical(values=postulaciones['weekday'].values, ordered=True).rename_categories(days)
postulaciones['weekday'].value_counts().sort_index().plot(kind='bar', title='Postulaciones por dia de semana')
# Al hacer el gráfico, podemos ver que los fines de semana tienen muchas menos postulaciones
# y que jueves y viernes son los dias de la semana con menos postulaciones.

In [ ]:
#este df contiene solo los dias entre el 15 y 28 de cada mes
enero_febrero_igual_dias = pd.DataFrame(columns = ['date'])
enero_febrero_igual_dias['date'] = postulaciones['date'][(pd.to_datetime(postulaciones['date']).dt.day > 14) & (pd.to_datetime(postulaciones['date']).dt.day < 29)]

In [ ]:
hist_enero_edit = pd.to_datetime(enero_febrero_igual_dias['date'][pd.to_datetime(enero_febrero_igual_dias['date']).dt.month == 1]).dt.day.hist(bins=13,color= 'lightgreen');
hist_enero_edit.set_title('Distribucion de postulaciones durante Enero(15-28)',fontsize=15)
hist_enero_edit.set_xlabel('Dias')
hist_enero_edit.set_ylabel('Cantidad de postulaciones');

In [ ]:
hist_Febrero_edit = pd.to_datetime(enero_febrero_igual_dias['date'][pd.to_datetime(enero_febrero_igual_dias['date']).dt.month == 2]).dt.day.hist(bins=13,color='lightblue');
hist_Febrero_edit.set_title('Distribucion de postulaciones durante Febrero(15-28)',fontsize=15)
hist_Febrero_edit.set_xlabel('Dias')
hist_Febrero_edit.set_ylabel('Cantidad de postulaciones');

In [ ]:
plt.hist(pd.to_datetime(enero_febrero_igual_dias['date']
        [pd.to_datetime(enero_febrero_igual_dias['date']).dt.month == 2]).dt.day, bins=13,
         color='lightblue',label= 'Febrero')
plt.hist(pd.to_datetime(enero_febrero_igual_dias['date']
        [pd.to_datetime(enero_febrero_igual_dias['date']).dt.month == 1]).dt.day,bins=13,
         color='violet',alpha=0.5, label='Enero')
plt.xlabel('Fechas')
plt.ylabel('Cantidad de postulaciones')
plt.title('Distribucion de las postulaciones entre el 15 y 28 de cada mes')
plt.legend()
plt.show()

De esta última comparación se aprecia que el número de postulaciones es en general
mayor para febrero, y que el último día de febrero presenta un pico muy alto en 
las postulaciones.

In [ ]:
educacion_postulantes.head()

In [ ]:
post_educ_estado = educacion_postulantes['estado'].value_counts().sort_index().plot(kind='bar')
post_educ_estado.set_title('Distribucion de postulaciones segun estado de la Educacion',fontsize=15)
post_educ_estado.set_xlabel('Estado')
post_educ_estado.set_ylabel('Cantidad de postulaciones');

In [ ]:
post_educ_carrera = np.log(educacion_postulantes['nombre'].value_counts() + 1).plot(kind='barh')
post_educ_carrera.set_title('Distribucion de postulaciones (log) segun la carrera',fontsize=15)
post_educ_carrera.set_ylabel('Nombre de la carrera')
post_educ_carrera.set_xlabel('Cantidad de postulaciones (log)');

In [ ]:
estado_y_carreras = educacion_postulantes[['nombre','estado']]
benchs_number = 7
benchs_names = ['Doctorado','Master','Otro','Posgrado','Secundario', 'Terciario/Tecnico','Universitario']

arch_en_curso = estado_y_carreras[estado_y_carreras['estado'] == 'En Curso']
arch_graduado = estado_y_carreras[estado_y_carreras['estado'] == 'Graduado']
arch_abandonado = estado_y_carreras[estado_y_carreras['estado'] == 'Abandonado']

#Uso una escala logaritmica asi se pueden ver mejor los datos en el grafico
en_curso = np.log(arch_en_curso['nombre'].value_counts() +1).sort_index()
abandonado = np.log(arch_abandonado['nombre'].value_counts() +1).sort_index()
graduado = np.log(arch_graduado['nombre'].value_counts() +1).sort_index()

index = np.arange(len(benchs_names))

bar_width = 0.2
opacity = 0.5

#plt.barh(index, en_curso, bar_width, alpha=0.7,color='y',label='En Curso')
plt.barh(index+bar_width, abandonado, bar_width, alpha=0.7,color='r',label='Abandonado')
plt.barh(index+2*bar_width, graduado, bar_width, alpha=0.7,color='g',label='Graduado')

plt.grid(True, axis='x', linewidth=0.2)
plt.title('Distribucion de postulaciones (log) segun el estado de cada carrera',fontsize = 15)
plt.ylabel('Carreras')
plt.xlabel('Cantidad de postulaciones (log)')
plt.yticks(index + 2*bar_width, benchs_names)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:

genero_edad_postulantes['edad'] = pd.to_datetime(genero_edad_postulantes['fechanacimiento'],errors='coerce').dt.year
genero_edad_postulantes['edad'] = 2018 - genero_edad_postulantes['edad']
genero_edad_postulantes.sort_values(by='edad', ascending=True)

In [ ]:
genero_edad_postulantes.fillna(0)
edad_postulantes = pd.DataFrame(columns=['edad'])
edad_postulantes['edad'] = genero_edad_postulantes['edad'][(genero_edad_postulantes['edad'] > 17) & (genero_edad_postulantes['edad'] < 71)]
edad_postulantes['edad'] = edad_postulantes['edad'].astype(int)
edad_postulantes.sort_values(by='edad', ascending = True)

In [ ]:
genero_edad_postulantes = genero_edad_postulantes.fillna(0)
genero_edad_postulantes['edad'] = genero_edad_postulantes['edad'].astype(int)
genero_edad_postulantes = genero_edad_postulantes[(genero_edad_postulantes['edad'] <71) & (genero_edad_postulantes['edad'] > 17)]
genero_edad_postulantes.head()

In [ ]:
post_genero = genero_edad_postulantes['sexo'].value_counts().plot(kind='barh')
post_genero.set_title('Distribucion de postulaciones segun el genero',fontsize=15)
post_genero.set_ylabel('Genero')
post_genero.set_xlabel('Cantidad de postulaciones');

In [ ]:
post_edad = edad_postulantes['edad'].value_counts().sort_index().plot(kind='line',color='green')
post_edad.set_title('Distribucion de postulaciones segun la edad de los postulantes',fontsize=15)
post_edad.set_xlabel('Edad')
post_edad.set_ylabel('Cantidad de postulaciones');



In [ ]:
femenino = pd.DataFrame(columns = ['genero','edad'])
masculino = pd.DataFrame(columns = ['genero','edad'])
no_declara = pd.DataFrame(columns = ['genero','edad'])

femenino = genero_edad_postulantes[genero_edad_postulantes['sexo'] == 'FEM']
masculino = genero_edad_postulantes[genero_edad_postulantes['sexo'] == 'MASC']
no_declara = genero_edad_postulantes[genero_edad_postulantes['sexo'] == 'NO_DECLARA']

#plt.hist(femenino['edad'], bins=53, color='pink',label= 'Postulantes Fememinas')
#plt.hist(masculino['edad'], bins=53, color='darkblue',alpha=0.3, label='Postulantes Masculinos')
#plt.hist(no_declara['edad'], bins=53, color='red',alpha=0.5, label='Postulantes sin sexo declarado')

plt.xlabel('Edad')
plt.ylabel('Cantidad de postulaciones')
plt.title('Distribucion de las postulaciones segun edad y el genero')
plt.legend()
plt.show()

In [ ]:
#Cantidad de postulaciones de personas que no declaran su sexo segun su edad.
no_declara['edad'].value_counts().head()

In [ ]:
avisos_detalle.head()

In [ ]:
#avisos_nivel_laboral = avisos_detalle['nivel_laboral'].value_counts().plot(kind='bar')
#avisos_nivel_laboral.set_title('Distribucion de los avisos laborales segun el nivel laboral pedido',fontsize=15)
#avisos_nivel_laboral.set_xlabel('Cantidad de avisos')
#avisos_nivel_laboral.set_ylabel('Nivel laboral');

avisos_detalle['nivel_laboral'].value_counts()

In [ ]:
avisos_tipo_de_trabajo = np.log(avisos_detalle['tipo_de_trabajo'].value_counts()+1).plot(kind='barh')
avisos_tipo_de_trabajo.set_title('Distribucion de los avisos (log) laborales segun el tipo de trabajo pedido',fontsize=15)
avisos_tipo_de_trabajo.set_ylabel('Tipo de trabajo')
avisos_tipo_de_trabajo.set_xlabel('Cantidad de avisos (log)');

avisos_detalle['tipo_de_trabajo'].value_counts().sum()

In [ ]:
avisos_zona = np.log(avisos_detalle['nombre_zona'].value_counts() + 1).sort_index().plot(kind='bar')
avisos_zona.set_title('Distribucion de los avisos laborales segun la zona de trabajo',fontsize=15)
avisos_zona.set_xlabel('Zona')
avisos_zona.set_ylabel('Cantidad de avisos (log)');

avisos_detalle['nombre_zona'].value_counts()

# Merge de los datos
recuerdo la forma de los datasets

In [ ]:
#vistas = vistas.set_index()
vistas['idpostulante'].value_counts()

In [ ]:
educacion_postulantes.head(15)

In [ ]:
postulantes = genero_edad_postulantes.set_index('idpostulante')
postulantes['anuncios_vistos'] = vistas['idpostulante'].value_counts()
postulantes['anuncios_vistos'] = postulantes['anuncios_vistos'].fillna(0)
#sobresimplificación horrible, habría que ponderar más fuerte estudios avanzados y estudios completos.
postulantes['nivel_educativo'] = educacion_postulantes['idpostulante'].value_counts()

postulantes.head(10)

In [ ]:
hist_educacion = postulantes['nivel_educativo'].hist(log=True, bins=9, label='cantidad de postulantes por nivel educativo')
hist_educacion.set_title('Cantidad de postulantes por nivel educativo')
hist_educacion.set_xlabel('Nivel educativo')
hist_educacion.set_ylabel('Cantidad de postulantes(log)')

In [ ]:
scatter_anuncios_vistos = postulantes.plot.scatter('nivel_educativo','anuncios_vistos', logy=True,alpha=0.3)
scatter_anuncios_vistos.set_title('Cantidad de anuncios vistos por nivel educativo')
scatter_anuncios_vistos.set_xlabel('Nivel educativo')
scatter_anuncios_vistos.set_ylabel('Cantidad de anuncios vistos por postulante(log)')

In [ ]:
bar_avg_anuncios_vistos = postulantes.groupby('nivel_educativo').agg(np.average).plot()
bar_avg_anuncios_vistos.set_title('Promedio de anuncios vistos por nivel educativo')
bar_avg_anuncios_vistos.set_xlabel('Nivel educativo')
bar_avg_anuncios_vistos.set_ylabel('Promedio de anuncios vistos')


In [ ]:
postulaciones['anuncios_vistos'] = vistas['idpostulante'].value_counts()
postulaciones.head()

In [ ]:
avisos_online.head()

In [ ]:
educacion_postulantes.head()

In [ ]:
#MERGE PARA VER LOS AVISOS ONLINE Y OFFLINE VISTOS POR POSTULANTES
avisos_vistos_por_postulantes = pd.merge(vistas, genero_edad_postulantes, on='idpostulante', how='inner')
avisos_vistos_por_postulantes.groupby('idpostulante').agg(np.sum)

In [ ]:
#RENOMBRO LA COLUMNA 'idAviso' DEL DF VISTAS PARA QUE COINCIDA CON EL DE AVISOS_ONLINE
vistas = vistas.rename(columns={'idAviso':'idaviso'})

In [ ]:
#HAGO UN MERGE PARA VER SOLO LOS AVISOS ONLINE QUE HALLAN SIDO VISTOS
avisos_online_vistos = pd.merge(vistas, avisos_online, on='idaviso', how='inner')

In [ ]:
#Hacer un merge con con los datos de el estado de las carreras y los sexos
postulantes_educacion_genero = pd.merge(genero_edad_postulantes, educacion_postulantes, on='idpostulante', how='inner')
postulantes_educacion_genero.head()